In [1]:
from cloudvolume import CloudVolume
import imageio

In [4]:
RES = [320, 320, 50]
# Z = [297,425,466,534,566,700,759,864,928,1159,1218,1288,1800,1801,1829,1965,2020,2061,2310,2824,2830,2842,2879]
Z = [429,2842]

IMG_CV = CloudVolume("precomputed://gs://dacey-human-retina-001-alignment-temp/rigid/img", mip=RES, fill_missing=True, progress=False)
DEFECT_CV = CloudVolume("precomputed://gs://dacey-human-retina-001-alignment-temp/rigid/defect_mask", mip=RES, fill_missing=True, progress=False)
# MISD1_CV = CloudVolume("precomputed://gs://dacey-human-retina-001-alignment-temp/pairwise/misd_raw/z-1", mip=RES, fill_missing=True, progress=False)

In [3]:
import numpy as np
for z in Z:
    img = IMG_CV[:,:,z]
    defect = DEFECT_CV[:,:,z]
    imageio.imwrite(f"defects/img_z{z}.png", img.squeeze().T)
    imageio.imwrite(f"defects/defect_z{z}.png", defect.squeeze().T)

In [6]:
from copy import deepcopy
DEFECT2_CV = CloudVolume("precomputed://gs://dacey-human-retina-001-alignment-temp/pairwise/misd_manual/z-1", mip=RES, fill_missing=True, progress=False, info=deepcopy(DEFECT_CV.info))
DEFECT2_CV.commit_info()
for z in Z:
    try:
        defect = imageio.imread(f"defects/defect_z{z}.png")[:,:,0].squeeze().T
    except IndexError:
        defect = imageio.imread(f"defects/defect_z{z}.png").squeeze().T
    DEFECT2_CV[:,:,z,0] = defect.reshape(1024,1024,1,1)
    DEFECT2_CV[:,:,z+1,0] = defect.reshape(1024,1024,1,1)

/tmp/ipykernel_968/1320649237.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  defect = imageio.imread(f"defects/defect_z{z}.png")[:,:,0].squeeze().T


In [27]:
IMG_CV = CloudVolume("precomputed://gs://dacey-human-retina-001-alignment-temp/rigid/img", mip=RES, fill_missing=True)
TISSUE_CV = CloudVolume("precomputed://gs://dacey-human-retina-001-alignment-temp/rigid/tissue_mask", mip=RES, fill_missing=True, info=deepcopy(DEFECT_CV.info))
TISSUE_CV.commit_info()

img = IMG_CV[:,:,:] > 0
TISSUE_CV[:,:,:] = img.astype(np.uint8)*255

Uploading: 100%|██████████| 4000/4000 [00:45<00:00, 87.50it/s] 


In [4]:
RES = [640, 640, 50]
from cloudvolume import CloudVolume
from copy import deepcopy
import numpy as np
DEFECT_CV = CloudVolume("gs://dacey-human-retina-001-alignment-temp/rigid/defect_mask_refine", mip=RES, fill_missing=True, progress=False)
TISSUE_CV = CloudVolume("precomputed://gs://dacey-human-retina-001-alignment-temp/rigid/tissue_mask", mip=RES, fill_missing=True, progress=False)
TISSUE_NO_DEFECT_CV = CloudVolume("precomputed://gs://dacey-human-retina-001-alignment-temp/rigid/tissue_no_defect_mask", mip=RES, fill_missing=True, progress=False)
# TISSUE_NO_DEFECT_CV.commit_info()

defects = DEFECT_CV[:,:,:] > 0
tissue = TISSUE_CV[:,:,:] > 0

tissue_no_defect = tissue & ~defects
TISSUE_NO_DEFECT_CV[:,:,:] = tissue_no_defect.astype(np.uint8)*255